In [32]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime, timedelta
import requests
import openpyxl
from pandas_market_calendars import get_calendar
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout

# Funções

In [33]:
def workday() -> datetime:
    return get_calendar('BMF').schedule(
        data_ini=(datetime.now() - timedelta(days=90)).strftime("%d/%m/%Y"),
        data_fim=datetime.now().strftime("%d/%m/%Y")
    ).loc[::-1, ("market_open")].iloc[0].date()

# Main

In [34]:
USER = Path(os.path.expanduser("~"))

In [35]:
excel_path = USER / r"C:\Users\GiordanoBrunoGava\OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx"
dados_cotacao__aj_consolidado_excel = pd.read_excel(excel_path)

In [36]:
dados_cotacao__aj_consolidado_excel

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,2023-08-29,"241,85","114,62","10,4",NaN,"5,51",118403.0,"108,3",7962.0,13089.0,...,"88,5120125468","8,46864353782","82,2174457333","78,8253826777","7,69008387322","8,05401611571","9,66812798059","102,832004798","107,756899756","22,6660169515"
1,2023-08-30,"243,85","113,9","10,49",NaN,"5,5",117535.0,"107,4",7936.0,13199.0,...,"89,3710155114","8,56568007836","82,8797063316","79,1242966874","7,77707577224","8,04517526377","9,6366014763","103,577697939","108,44753261","22,3821002464"
2,2023-08-31,"247,65","112,31","10,71","144,377510948","5,52",115741.0,"104,2",7813.0,13437.0,...,"90,1085433093","8,59214458941","83,1799311361","79,1155050989","7,82927091165","8,06285696765","9,63222279515","104,332597169","106,04849217","21,5492779117"
3,2023-09-01,"246,8","114,05","10,65","153,278517416","5,66",117892.0,"107,24",7940.0,13390.0,...,"89,7651888209","8,61891139499","83,8157013104","79,1689675418","7,82047397804","8,11996440965","9,71191479209","104,595014702","106,496803456","21,7574834954"
4,2023-09-04,"246,94","114,14","10,69",NaN,"5,66",117776.0,"106,45",7946.0,13402.0,...,"91,0153513172","8,61891139499","84,9106388328","79,240250799","7,89964638052","8,11996440965","9,66725224436","104,873687033","107,17384499","21,7290918249"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2024-08-16,"338,35","130,29","14,91","228,05","5,88",133953.0,"102,99",9256.0,19343.0,...,"96,74","9,28","84,56","68,9","9,99","9,12","9,6","103,6","113,42","23,26"
245,2024-08-19,"338,08","131,98","14,92","216,83","5,88",135777.0,106,9423.0,19314.0,...,"96,41","9,2","85,17","69,85","9,99","9,12","9,45","103,5","113,43","23,42"
246,2024-08-20,"342,17","132,44","15,06","209,4","5,8",136087.0,"105,94",9446.0,19543.0,...,"96,64","9,21","85,29","69,74","9,99","9,12","9,46","103,21","113,01","23,37"
247,2024-08-21,"343,25","132,74","15,16",NaN,"5,89",136463.0,"105,6",9499.0,19583.0,...,"96,58","9,2","84,45","69,67","10,01","9,11","9,41",103,"112,98","23,34"


In [37]:
dados_cotacao__aj_consolidado_excel['Data'] = pd.to_datetime(dados_cotacao__aj_consolidado_excel['Data'], format="%d/%m/%Y")

In [38]:
ultima_linha_com_data = dados_cotacao__aj_consolidado_excel['Data'].last_valid_index()

In [39]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_1=(datetime.now() - timedelta(days=360)).strftime("%d/%m/%Y")
data_fim_1=(datetime.now() - timedelta(days=300)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_1}%26data_fim%3D{data_fim_1}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_1 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [40]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_2=(datetime.now() - timedelta(days=299)).strftime("%d/%m/%Y")
data_fim_2=(datetime.now() - timedelta(days=240)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_2}%26data_fim%3D{data_fim_2}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_2 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [41]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_3=(datetime.now() - timedelta(days=239)).strftime("%d/%m/%Y")
data_fim_3=(datetime.now() - timedelta(days=180)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_3}%26data_fim%3D{data_fim_3}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_3 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [42]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_4=(datetime.now() - timedelta(days=179)).strftime("%d/%m/%Y")
data_fim_4=(datetime.now() - timedelta(days=120)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_4}%26data_fim%3D{data_fim_4}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_4 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [43]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_5=(datetime.now() - timedelta(days=119)).strftime("%d/%m/%Y")
data_fim_5=(datetime.now() - timedelta(days=60)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_5}%26data_fim%3D{data_fim_5}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_5 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [45]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_6=(datetime.now() - timedelta(days=59)).strftime("%d/%m/%Y")
data_fim_6=datetime.now().strftime("%d/%m/%Y") 

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_6}%26data_fim%3D{data_fim_6}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_6 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=6000)

In [46]:
data_1 = list()
for lin in (table := response_1.json()["tables"]["tab0"]):
    data_1.append(table[lin])

In [47]:
dados_cotacao__aj_novos_1 = pd.DataFrame(data_1)

In [48]:
data_2 = list()
for lin in (table := response_2.json()["tables"]["tab0"]):
    data_2.append(table[lin])

In [49]:
dados_cotacao__aj_novos_2 = pd.DataFrame(data_2)

dados_cotacao__aj_novos_2 = dados_cotacao__aj_novos_2.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_2.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,03/11/2023,"236,98","114,75","10,31","166,886344649","5,27",118159,100,7996,12964,...,"87,6675513869","8,51349478466","84,1712918067","79,523165789","8,19855171339","8,5432116505","9,81702831912","102,169222293","109,093324717","22,6092336105"
1,06/11/2023,"236,25","114,81","10,35","168,461026193","5,28",118431,"98,25",8058,12974,...,"87,3615250268","8,59570664417","84,3694690776","78,4791972883","8,13541655168","8,53402540141","9,81613170497","101,886596506","109,782433213","22,3253169054"
2,07/11/2023,"236,78","115,67","10,41","165,640552288","5,26",119268,"100,6",8100,13014,...,"87,397528128","8,61397594628","84,4055013087","78,3974954056","8,1444358605","8,53402540141","10,0725633516","102,687369571","109,307507087","22,8931503155"
3,08/11/2023,"238,2","115,52","10,47","167,544123015","5,26",119176,"101,1",8092,13149,...,"87,3885273527","8,61397594628","84,4145093665","78,6153670927","8,12639724287","8,50646665415","9,86275564073","102,677948711","108,851205516","22,9972531073"
4,09/11/2023,"237,95","115,18","10,46","167,773348809","5,22",119034,"99,71",8076,13096,...,"87,7755606905","8,60484129523","84,4865738286","78,8060048189","8,1444358605","8,55239789959","9,86275564073","102,593160975","108,041037419","22,7417280728"
5,10/11/2023,"240,15","117,01","10,63","167,464392304","5,18",120568,"102,09",8187,13262,...,"88,0725862752","8,60484129523","84,4955818864","79,1418903365","8,15345516932","8,61670164319","9,86275564073","101,622812437","108,264532067","23,375808714"
6,13/11/2023,"240,4","116,82","10,59","166,856445632","5,24",120410,"101,68",8181,13208,...,"87,973577747","8,62311059734","84,6757430418","78,542743197","8,17149378695","8,6810053868","9,86275564073","101,698179314","108,944328286","23,3284892632"
7,14/11/2023,"242,75","119,61","10,73","166,587354482","5,32",123165,"104,75",8356,13412,...,"87,9645769717","8,59570664417","85,414403779","78,1977796925","8,18051309576","8,67181913771","9,86544548318","101,745283612","108,7767073","23,9341782339"
8,16/11/2023,"243,65","120,96","10,74","166,617253499","5,23",124639,"106,2",8471,13390,...,"88,2075979047","8,61397594628","84,9099525438","78,2704035882","8,11737793405","8,63507414136","10,038492014","101,886596506","108,488026714","24,1707754881"
9,17/11/2023,"245,85","121,1","10,83","168,102237993","5,28",124773,"105,8",8458,13485,...,"89,0716723331","8,65051455051","84,991894381","78,5245872231","8,17149378695","8,63507414136","10,0232495734","102,197484872","108,385591667","23,9625699044"


In [50]:
data_3 = list()
for lin in (table := response_3.json()["tables"]["tab0"]):
    data_3.append(table[lin])

In [51]:
dados_cotacao__aj_novos_3 = pd.DataFrame(data_3)

dados_cotacao__aj_novos_3 = dados_cotacao__aj_novos_3.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_3.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,02/01/2024,259,"128,76","11,33","235,095967985","4,93",132696,"109,91",8994,14024,...,"91,4156240962","8,91286447742","85,7325725496","79,9450721371","8,63288100387","8,71811118217","10,2438600909","104,021416929","111,104314139","24,3695171816"
1,03/01/2024,"257,1","128,99","11,22","235,50458788","5,01",132833,"109,9",9015,13925,...,"91,2594679859","8,92224644002","85,7785960479","80,4677719666","8,56748039021","8,67134019943","10,3255466376","104,183905589","112,702869314","24,3695171816"
2,04/01/2024,"254,9","127,35","11,13","234,278728197","4,93",131225,"107,79",8930,13829,...,"91,562594553","8,91286447742","85,2263140688","81,8491929445","8,56748039021","8,63392341324","10,371437956","104,413301344","112,031286962","23,8679310027"
3,05/01/2024,"253,95","128,25","11,09","234,019603386","4,87",132022,"108,85",8963,13795,...,"91,562594553","8,95039232785","85,6037067545","81,4291662958","8,5581374454","8,65263180634","10,3071901102","105,063255984","112,910965254","23,8773948929"
4,08/01/2024,"257,11","128,57","11,3","234,886674869","4,8",132426,"110,61",8970,14035,...,"91,6728223955","8,96915625306","85,3367704646","81,6345126574","8,53945155578","8,61521502015","10,912955513","105,13016308","112,277218528","24,1518477078"
5,09/01/2024,"258,85","127,59","11,42",,"4,77",131446,"110,34",8909,14088,...,"91,8106071987","8,96915625306","84,9685824786","81,1864842321","8,56748039021","8,6058608236","10,9221337767","105,044139671","112,352889778","24,3221977308"
6,10/01/2024,"259,28","126,94","11,45","235,195631374","4,76",130841,"109,6",8856,14153,...,"91,9483920019","8,96915625306","85,1894952702","81,2051520832","8,53945155578","8,58715243051","10,9037772493","105,044139671","112,144793838","24,2938060603"
7,11/01/2024,"258,2","126,81","11,42","232,484787197","4,79",130648,"109,22",8842,14111,...,"92,0035059232","8,97853821567","85,5208644576","81,2051520832","8,53945155578","8,51231885813","10,968025095","105,034581515","111,993451337","24,0098893553"
8,12/01/2024,"257,48","127,23","11,39","229,883572748","4,76",130987,"109,7",8847,14078,...,"91,6728223955","8,95977429046","85,3827939629","82,0265375295","8,52076566616","8,57779823396","10,7936380852","104,891209168","112,0880404","24,1991671586"
9,15/01/2024,"258,11","127,82","11,46",,"4,75",131520,"109,7",8883,14208,...,"91,764678931","8,92224644002","85,1434717719","81,9518661253","8,50207977654","8,55908984086","11,1883034233","106,372723422","112,466396655","24,1897032684"


In [52]:
data_4 = list()
for lin in (table := response_4.json()["tables"]["tab0"]):
    data_4.append(table[lin])

In [53]:
dados_cotacao__aj_novos_4 = pd.DataFrame(data_4)

dados_cotacao__aj_novos_4 = dados_cotacao__aj_novos_4.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_4.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,04/03/2024,"282,34","124,5","12,57",,"4,86",128340,"105,65",8764,15655,...,"95,4272646355","9,09193166821","88,4089249525","82,2790715339","9,07150610541","8,59716515625","10,8008418296","107,792705327","113,745151771","22,978325327"
1,05/03/2024,"279,86","124,4","12,37","237,891261285","4,83",128098,"106,15",8753,15467,...,"95,0594750001","9,09193166821","87,5340547652","82,6424189735","9,1099446906","8,59716515625","10,3047893019","108,510095397","114,235598402","22,864758645"
2,06/03/2024,"280,42","125,15","12,43","236,973686405","4,88",128890,"106,7",8758,15529,...,"95,0783360071","9,10151220738","87,8633069863","82,2217008855","9,1099446906","8,56857148057","10,5387763433","108,374372951","113,831701177","23,091892009"
3,07/03/2024,"282,62","124,67","12,59","232,874520365","4,84",128339,"106,16",8762,15717,...,"95,4555561459","9,08235112905","87,468204321","82,451183479","9,1099446906","8,59716515625","10,557679978","108,481012016","113,956716984","22,9499336565"
4,08/03/2024,"283,75","123,26","12,52","236,325399805","4,9",127070,107,8686,15745,...,"95,6253052084","9,11109274655","87,9573790494","82,0017800668","8,96579999613","8,62575883194","10,58603543","108,413150793","114,427930413","22,9499336565"
5,11/03/2024,"283,25","122,32","12,47","234,031462605",5,126123,"106,16",8620,15642,...,"95,9176508161","9,07277058988","87,6845700663","82,030465391","8,97540964243","8,59716515625","10,6332945168","107,608510579","113,995183387","22,8363669745"
6,12/03/2024,"286,24","123,81","12,63","232,385812005","5,1",127667,"107,65",8677,15897,...,"96,0025253473","9,03444843322","87,8162709547","81,5141295556","9,04267716652","8,61622760671","10,557679978","107,608510579","113,475886955","23,1581392402"
7,13/03/2024,"285,57","124,37","12,54","202,933653084","5,13",128006,"108,1",8657,15830,...,"96,3797454862","9,05360951155","87,8633069863","82,2217008855","9,1291639832","8,5781027058","10,3780954484","107,608510579","113,764384973","23,2433142517"
8,14/03/2024,"285,75","123,82","12,54","183,704475164","5,05",127689,"108,05",8597,15860,...,"96,2854404515","9,02486789405","87,9479718431","82,2121391108","9,1291639832","8,58763393103","10,0189263891","107,278898925","113,572052961","23,1865309107"
9,15/03/2024,"284,4","122,87","12,42","193,698062444","5,07",126741,"106,67",8561,15761,...,"96,3420234723","8,98654573739","88,5755603545","81,7340503744","9,19643150729","8,63529005717","9,54633552174","104,031254686","114,2163652","23,3379531534"


In [54]:
data_5 = list()
for lin in (table := response_5.json()["tables"]["tab0"]):
    data_5.append(table[lin])

In [55]:
dados_cotacao__aj_novos_5 = pd.DataFrame(data_5)

dados_cotacao__aj_novos_5 = dados_cotacao__aj_novos_5.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_5.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,02/05/2024,288,"123,35","12,5","188,092876764","5,73",127122,"101,8",8842,16141,...,"97,0078277078","9,1314893617","86,7890248108","68,7390612942","9,65026530612","8,75367015123","9,03191819912","103,256656025","111,660335127","20,27"
1,03/05/2024,"289,55","124,95","12,64","185,789953654","5,72",128508,105,8962,16263,...,"97,842095026","9,18042553191","87,1542903931","69,0426098555","9,67","8,81196318221","9,23262749243","103,236982469","112,881040208","20,99"
2,06/05/2024,"292,1","124,71","12,8","191,390306349","5,72",128465,"103,1",8943,16449,...,"97,6868825017","9,1314893617","88,2308626357","71,4220389003","9,62066326531","8,7633856564","9,17528198006","102,804164249","112,793149442","20,93"
3,07/05/2024,"292,25","125,4","12,79","193,516643468","5,66",129210,"103,6",8914,16421,...,"97,5413707601","9,12170212766","87,4714947146","71,5297496801","9,61079591837","8,75367015123","9,12749405308","103,089430803","112,646664832","21,08"
4,08/05/2024,"293,4","125,75","12,82","195,283599666","5,63",129480,"104,07",8937,16481,...,"97,3958590186","9,15106382979","87,0293311149","70,4918094383","9,61079591837","8,78281666672","9,28997300481","103,227145692","113,457213006","21,12"
5,09/05/2024,"298,3","124,52","12,99",,"5,83",128188,"101,75",8837,16680,...,"97,0757331871","9,1314893617","86,5102694979","69,5615799763","9,61079591837","8,7633856564","9,22306990704","103,109104359","113,6720571","20,59"
6,10/05/2024,"299,35","124,01","13,06","196,126005089","5,89",127599,"100,65",8789,16749,...,"97,124237101","9,16085106383","86,6448410283","69,52241242","9,61079591837","8,83139419253","9,36643368798","103,266492803","114,531433477","21,2"
7,13/05/2024,"299,1","124,56","13,05","195,986122055","5,99",128154,"100,91",8871,16745,...,"96,8041112696","9,2","86,5198817501","69,6203313108","9,5417244898","8,81196318221","9,21351232164","102,076242695","114,414245789","21,37"
8,14/05/2024,"299,3","124,81","13,1",,"5,95",128515,101,8890,16819,...,"97,1824417976","9,17063829787","86,8755350803","70,2078446552","9,53185714286","8,80224767704","9,16572439466","101,909017473","113,759947866","21,37"
9,15/05/2024,"303,2","124,23","13,33",,"6,01",128027,"102,2",8908,17063,...,"97,1533394493","9,19021276596","86,4333714806","69,6692907561","9,55159183673","8,86054070802","9,17528198006","102,548408027","113,945495038","22,13"


In [56]:
data_6 = list()
for lin in (table := response_6.json()["tables"]["tab0"]):
    data_6.append(table[lin])

In [57]:
dados_cotacao__aj_novos_6 = pd.DataFrame(data_6)

dados_cotacao__aj_novos_6 = dados_cotacao__aj_novos_6.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_6.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,01/07/2024,"344,9","121,08","15,58","216,369078414","6,05",124718,"96,35",8796,20046,...,"96,1834864643","9,17032258065","87,7767726161","65,99","9,66","8,89395814978","9,45906975677","98,3561062458","107,047767369","21,08"
1,02/07/2024,"347,01","121,33","15,81","217,128443455","6,11",124787,"96,78",8805,20245,...,"95,5104593929","9,16043010753","86,9805891243","65,9","9,7","8,87414977974","9,44928788526","97,5521865076","106,869172521","20,77"
2,03/07/2024,342,"122,14","15,67",,"6,09",125661,"98,53",8841,20104,...,"96,0053322395","9,19010752688","87,2459836216","65,88","9,74","8,86424559471","9,1851773543","98,0087335194","105,301506635","21,24"
3,04/07/2024,"337,92","122,62","15,51",,"5,95",126163,"100,63",8909,19946,...,"95,8568703856","9,20989247312","87,10837166","65,97","9,65","8,88405396476","9,2438685834","99,646347801","107,752224824","21,85"
4,05/07/2024,"338,51","122,7","15,53",,"5,9",126267,"101,5",8895,19884,...,"96,6387694832","9,2","88,4156852948","66,38","9,8","8,94347907489","9,34168729857","100,738090655","107,573629977","22,2"
5,08/07/2024,"339,67","122,94","15,59","213,051852183","5,86",126548,"100,9",8904,19919,...,"96,8367186218","9,2","87,8554080227","66,85","9,86","8,95338325991","9,46885162829","102,038257146","106,015886027",22
6,09/07/2024,"336,22","123,54","15,44",,"5,88",127108,"102,28",8929,19770,...,"96,3220508614","9,2","88,4255147206","67,19","9,85","8,96328744493","9,41994227071","102,177206236","106,521904762","22,13"
7,10/07/2024,"339,65","123,5","15,59",,"5,83",127218,102,8938,19993,...,"96,9158982773","9,2","88,1306319458","68,22","9,9","8,95338325991","9,34168729857","102,226830911","107,821678376","21,9"
8,11/07/2024,"338,5","124,57","15,33",,"5,99",128293,"102,82",9035,19727,...,"97,9452337982","9,2","87,7767726161","68,3","9,9","8,993","9,32212355554","102,524578963","108,674964871","21,9"
9,12/07/2024,"339,87","125,2","15,4",,"6,04",128896,"102,7",9071,19781,...,"97,9848236259","9,22967741935","87,7767726161",68,"9,88","9,00290418502","9,41994227071","102,703227793","109,895362998","22,1"


In [58]:
dados_cotacao__aj_novos = pd.concat([dados_cotacao__aj_novos_1, dados_cotacao__aj_novos_2, dados_cotacao__aj_novos_3, dados_cotacao__aj_novos_4, dados_cotacao__aj_novos_5, dados_cotacao__aj_novos_6], ignore_index=True) 

In [59]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(
    columns = {
        # Dict comprehension para colocar os nomes antigos da coluna com os valores da primeira linha.
        nome_antigo_da_coluna: nome_novo_da_coluna for
            nome_antigo_da_coluna, nome_novo_da_coluna in
                # Zip para poder fazer unpack das duas informacoes
                zip(
                    list(dados_cotacao__aj_novos.columns), [
                        # Como os dados da primeira linha vem em formatod indeseja ja formatamos aqui
                        str(item).replace("\nPRECO_AJ(,,,A,C)", "") for
                            item in
                                # iloc para pegar a primeira linha e todas as colunas
                                list(dados_cotacao__aj_novos.iloc[0, :])
                    ]
                )
        }
)

In [60]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(index = 0)

In [61]:
# ARRUMA AS DATAS ERRADAS PARA O FORMATO DAS OUTRAS
parser.parse("12-06-2024", dayfirst = True)

datetime.datetime(2024, 6, 12, 0, 0)

In [62]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,04/09/2023,"246,94","114,14","10,69",,"5,66",117776,"106,45",7946,13402,...,"91,0153513172","8,61891139499","84,1691907116","79,240250799","7,89964638052","8,11996440965","9,66725224436","104,873687033","107,17384499","21,7290918249"
2,05/09/2023,"248,1","113,62","10,77","154,841669559","5,6",117331,105,7912,13519,...,"89,7123650534","8,61891139499","85,5959355209","79,3917277206","7,82047397804","8,11996440965","9,62959558646","104,678616401","107,457470498","21,624989033"
3,06/09/2023,"246,46","112,25","10,67","154,841669559","5,6",115985,"104,31",7841,13423,...,"89,9412680457","8,65460046909","84,0291421414","78,8214616629","7,91724024774","8,11996440965","9,66287356321","104,595014702","107,939834006","21,3316084378"
4,08/09/2023,"246,35","111,66","10,62",,"5,44",115313,"103,55",7814,13364,...,"90,5839572164","8,66352273761","85,5959355209","79,3828173134","7,92603718135","8,14673352309","9,69965448487","105,24525014","108,59173331","21,3883917788"
5,11/09/2023,"244,9","113,48","10,62","150,650032285","5,46",116883,"105,1",7914,13423,...,"90,495917604","8,65460046909","85,3420974873","80,3273204715","8,00520958382","8,13781048528","9,69440006749","105,040890431","107,361388145","21,8710501774"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,22/08/2024,"347,31","131,57","15,17",,"5,94",135173,"103,14",9419,19743,...,"96,53","9,23","84,24","69,35","9,99","9,1","9,36","102,74","112,6","23,08"
247,23/08/2024,"344,48","131,98","15,05",,"5,91",135608,"105,85",9481,19585,...,"96,9","9,24","84,39","69,5",10,"9,11","9,42","103,1","112,67","23,86"
248,26/08/2024,"343,9","133,21","14,95","207,06","5,79",136888,"105,78",9520,19413,...,"96,86","9,19",84,"69,66","10,06","9,1","9,39","103,08","112,35","23,93"
249,27/08/2024,"345,58","133,01","15,08",,"5,82",136775,"104,75",9466,19529,...,"97,31","9,19","84,23","69,5","10,01","9,1","9,37","103,08","111,91","23,7"


In [63]:
dados_cotacao__aj_novos.reset_index(drop = True)

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,04/09/2023,"246,94","114,14","10,69",,"5,66",117776,"106,45",7946,13402,...,"91,0153513172","8,61891139499","84,1691907116","79,240250799","7,89964638052","8,11996440965","9,66725224436","104,873687033","107,17384499","21,7290918249"
1,05/09/2023,"248,1","113,62","10,77","154,841669559","5,6",117331,105,7912,13519,...,"89,7123650534","8,61891139499","85,5959355209","79,3917277206","7,82047397804","8,11996440965","9,62959558646","104,678616401","107,457470498","21,624989033"
2,06/09/2023,"246,46","112,25","10,67","154,841669559","5,6",115985,"104,31",7841,13423,...,"89,9412680457","8,65460046909","84,0291421414","78,8214616629","7,91724024774","8,11996440965","9,66287356321","104,595014702","107,939834006","21,3316084378"
3,08/09/2023,"246,35","111,66","10,62",,"5,44",115313,"103,55",7814,13364,...,"90,5839572164","8,66352273761","85,5959355209","79,3828173134","7,92603718135","8,14673352309","9,69965448487","105,24525014","108,59173331","21,3883917788"
4,11/09/2023,"244,9","113,48","10,62","150,650032285","5,46",116883,"105,1",7914,13423,...,"90,495917604","8,65460046909","85,3420974873","80,3273204715","8,00520958382","8,13781048528","9,69440006749","105,040890431","107,361388145","21,8710501774"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,22/08/2024,"347,31","131,57","15,17",,"5,94",135173,"103,14",9419,19743,...,"96,53","9,23","84,24","69,35","9,99","9,1","9,36","102,74","112,6","23,08"
246,23/08/2024,"344,48","131,98","15,05",,"5,91",135608,"105,85",9481,19585,...,"96,9","9,24","84,39","69,5",10,"9,11","9,42","103,1","112,67","23,86"
247,26/08/2024,"343,9","133,21","14,95","207,06","5,79",136888,"105,78",9520,19413,...,"96,86","9,19",84,"69,66","10,06","9,1","9,39","103,08","112,35","23,93"
248,27/08/2024,"345,58","133,01","15,08",,"5,82",136775,"104,75",9466,19529,...,"97,31","9,19","84,23","69,5","10,01","9,1","9,37","103,08","111,91","23,7"


In [ ]:
# dados_cotacao__aj_novos['Data'] = pd.to_datetime(dados_cotacao__aj_novos['Data'])

In [79]:
# Garantindo que todos os valores vazios sejam reconhecidos como NaN
dados_cotacao__aj_novos.replace('', np.nan, inplace=True)

# Verificando se todas as colunas, exceto a primeira, estão vazias
linhas_para_remover = dados_cotacao__aj_novos.iloc[:, 1:].isna().all(axis=1)

# Removendo as linhas onde todas as colunas, exceto a primeira, estão vazias
dados_cotacao__aj_novos = dados_cotacao__aj_novos[~linhas_para_remover]

In [80]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,04/09/2023,"246,94","114,14","10,69",NaN,"5,66",117776,"106,45",7946,13402,...,"91,0153513172","8,61891139499","84,1691907116","79,240250799","7,89964638052","8,11996440965","9,66725224436","104,873687033","107,17384499","21,7290918249"
2,05/09/2023,"248,1","113,62","10,77","154,841669559","5,6",117331,105,7912,13519,...,"89,7123650534","8,61891139499","85,5959355209","79,3917277206","7,82047397804","8,11996440965","9,62959558646","104,678616401","107,457470498","21,624989033"
3,06/09/2023,"246,46","112,25","10,67","154,841669559","5,6",115985,"104,31",7841,13423,...,"89,9412680457","8,65460046909","84,0291421414","78,8214616629","7,91724024774","8,11996440965","9,66287356321","104,595014702","107,939834006","21,3316084378"
4,08/09/2023,"246,35","111,66","10,62",NaN,"5,44",115313,"103,55",7814,13364,...,"90,5839572164","8,66352273761","85,5959355209","79,3828173134","7,92603718135","8,14673352309","9,69965448487","105,24525014","108,59173331","21,3883917788"
5,11/09/2023,"244,9","113,48","10,62","150,650032285","5,46",116883,"105,1",7914,13423,...,"90,495917604","8,65460046909","85,3420974873","80,3273204715","8,00520958382","8,13781048528","9,69440006749","105,040890431","107,361388145","21,8710501774"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,20/08/2024,"342,17","132,44","15,06","209,4","5,8",136087,"105,94",9446,19543,...,"96,64","9,21","85,29","69,74","9,99","9,12","9,46","103,21","113,01","23,37"
245,21/08/2024,"343,25","132,74","15,16",NaN,"5,89",136463,"105,6",9499,19583,...,"96,58","9,2","84,45","69,67","10,01","9,11","9,41",103,"112,98","23,34"
246,22/08/2024,"347,31","131,57","15,17",NaN,"5,94",135173,"103,14",9419,19743,...,"96,53","9,23","84,24","69,35","9,99","9,1","9,36","102,74","112,6","23,08"
247,23/08/2024,"344,48","131,98","15,05",NaN,"5,91",135608,"105,85",9481,19585,...,"96,9","9,24","84,39","69,5",10,"9,11","9,42","103,1","112,67","23,86"


In [81]:
if dados_cotacao__aj_consolidado_excel.index.duplicated().any():
    dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.reset_index(drop=True)
if dados_cotacao__aj_novos.index.duplicated().any():
    dados_cotacao__aj_novos = dados_cotacao__aj_novos.reset_index(drop=True)

In [82]:
dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.loc[:, ~dados_cotacao__aj_consolidado_excel.columns.duplicated()]
dados_cotacao__aj_novos = dados_cotacao__aj_novos.loc[:, ~dados_cotacao__aj_novos.columns.duplicated()]

In [83]:
# SALVA O DF CONCATENANDO NO EXCEL
output_path = USER / r'OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx'

dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(dados_cotacao__aj_novos.index[-1])
dados_cotacao__aj_novos.to_excel(output_path, index=False)

df = pd.read_excel(output_path)
df.replace(['-', '[]'], np.nan, inplace=True)
df = df.dropna(how='all', subset=df.columns.difference(['DATA']))
df.to_excel(output_path, index=False)


print('Dados atualizados e salvos no Excel.')
print('URL construída:', url + payload)

Dados atualizados e salvos no Excel.
URL construída: https://www.comdinheiro.com.br/Clientes/API/EndPoint001.phpusername=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D30/06/2024%26data_fim%3D28/08/2024%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3
